In [1]:
import mne
import glob
import os
import pandas as pd
import numpy as np
from pprint import pprint
import re

In [2]:
df = pd.read_hdf('chb02.hdf')

In [3]:
print(df)

            FP1-F7        FP2-F8     frame  window  test  seizure
0    -2.100122e-04  9.865690e-05       0.0     0.0     0        0
1     1.953602e-07  1.953602e-07       1.0     0.0     0        0
2     1.953602e-07  1.953602e-07       2.0     0.0     0        0
3     1.953602e-07  1.953602e-07       3.0     0.0     0        0
4     1.953602e-07  1.953602e-07       4.0     0.0     0        0
...            ...           ...       ...     ...   ...      ...
7675 -2.949939e-05  2.285714e-05  921595.0   238.0     2        0
7676 -1.777778e-05  1.621490e-05  921596.0   238.0     2        0
7677 -5.645910e-05  1.074481e-05  921597.0   238.0     2        0
7678 -3.262515e-05  1.699634e-05  921598.0   238.0     2        0
7679 -1.191697e-05  1.504274e-05  921599.0   238.0     2        0

[4147200 rows x 6 columns]


In [4]:
maximoRegistros=df['test'].max() #Para obtener el número mayor y saber cuantos registros hay en los archivos

In [5]:
iteracion=0
registros=[]
while iteracion<=maximoRegistros:
    registros.append(df[df['test'] == iteracion]) #Acomodar los valores en una lista 
    iteracion+=1

In [6]:
registros[0]

,FP1-F7,FP2-F8,frame,window,test,seizure
0,-2.100122e-04,9.865690e-05,0.0,0.0,0,0
1,1.953602e-07,1.953602e-07,1.0,0.0,0,0
2,1.953602e-07,1.953602e-07,2.0,0.0,0,0
3,1.953602e-07,1.953602e-07,3.0,0.0,0,0
4,1.953602e-07,1.953602e-07,4.0,0.0,0,0
...,...,...,...,...,...,...
7675,-6.446886e-06,-4.004884e-05,241915.0,61.0,0,0
7676,1.758242e-06,-4.239316e-05,241916.0,61.0,0,0
7677,-9.768010e-07,-4.590965e-05,241917.0,61.0,0,0
7678,4.884005e-06,-4.317460e-05,241918.0,61.0,0,0


In [7]:
registros[1]

,FP1-F7,FP2-F8,frame,window,test,seizure
0,0.000063,0.000277,0.0,0.0,1,0
1,0.000002,0.000002,1.0,0.0,1,0
2,-0.000009,-0.000009,2.0,0.0,1,0
3,-0.000027,-0.000019,3.0,0.0,1,0
4,0.000059,0.000066,4.0,0.0,1,0
...,...,...,...,...,...,...
7675,-0.000001,0.000013,921595.0,238.0,1,0
7676,-0.000009,0.000019,921596.0,238.0,1,0
7677,-0.000010,0.000019,921597.0,238.0,1,0
7678,-0.000007,0.000017,921598.0,238.0,1,0


In [8]:
registros[2]

,FP1-F7,FP2-F8,frame,window,test,seizure
0,-2.949939e-05,1.269841e-05,0.0,0.0,2,0
1,1.953602e-07,1.953602e-07,1.0,0.0,2,0
2,1.953602e-07,1.953602e-07,2.0,0.0,2,0
3,9.768010e-07,1.953602e-07,3.0,0.0,2,0
4,-9.768010e-07,5.860806e-07,4.0,0.0,2,0
...,...,...,...,...,...,...
7675,-2.949939e-05,2.285714e-05,921595.0,238.0,2,0
7676,-1.777778e-05,1.621490e-05,921596.0,238.0,2,0
7677,-5.645910e-05,1.074481e-05,921597.0,238.0,2,0
7678,-3.262515e-05,1.699634e-05,921598.0,238.0,2,0


In [29]:
iteracion=0
valorVentana=0.0
listaDeVentanasAtaques=[]
while iteracion<=maximoRegistros:
    maxVentana=registros[iteracion]['window'].max()
    print('--------------------Registro {iteracion}--------------------'.format(iteracion=iteracion+1))
    listaProvisional=[]
    while valorVentana<=maxVentana:
        valoresVentanaActual=registros[iteracion][registros[iteracion]['window'] == valorVentana]
        framesSeizure=valoresVentanaActual[valoresVentanaActual['seizure']==1]
        print('Ventana {valor}'.format(valor=valorVentana))
        tamañoVentana=len(valoresVentanaActual)#Número de frames de la ventana actual
        print('Frames en la ventana {framesTotales}'.format(framesTotales=tamañoVentana))
        tamañoframes=len(framesSeizure)#Número de frames con seizure=1
        print('Frames en la ventana con seizure=1 {framesTotales}'.format(framesTotales=tamañoframes))
        
        porcentajeVentana=tamañoVentana*0.5
        if(tamañoframes>=porcentajeVentana):
            print('Se trata de un ataque epiléptico')
            listaProvisional.append(valorVentana)
        else:
            print('No es ataque epiléptico')
        print('')
        valorVentana+=1
    listaDeVentanasAtaques.append(listaProvisional)
    valorVentana=0.0
    iteracion+=1

--------------------Registro 1--------------------
Ventana 0.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 1.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 2.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 3.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 4.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 5.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 6.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 7.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 1280
No es ataque epiléptico

Ventana 8.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 5120
Se trata de un ataque epiléptico

Ventana 9.0
Frames en la ventana 7

Ventana 41.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 42.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 43.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 44.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 45.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 46.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 47.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 48.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 49.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 50.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque ep

Ventana 123.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 124.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 125.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 126.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 127.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 128.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 129.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 130.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 131.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 132.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es

Ventana 228.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 229.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 230.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 231.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 232.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 233.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 234.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 235.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 236.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 237.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es

Ventana 70.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 71.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 72.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 73.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 74.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 75.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 76.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 77.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 78.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 79.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque ep

Ventana 176.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 177.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 178.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 179.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 180.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 181.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 182.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 183.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 184.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es ataque epiléptico

Ventana 185.0
Frames en la ventana 7680
Frames en la ventana con seizure=1 0
No es

In [30]:
listaDeVentanasAtaques[0]

[8.0, 9.0, 10.0, 11.0, 12.0, 13.0]

In [31]:
listaDeVentanasAtaques[1]

[198.0, 199.0, 200.0, 201.0, 202.0]

In [32]:
listaDeVentanasAtaques[2]

[]